 # Sofaescore web scraper 

#### OBJETIVO: obtener el performance de todos los jugaodres de LaLiga en los ultimos 29 partidos

#### DEPENDENCIAS:

In [25]:
import requests
import os
import regex 
import json
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException  # Importa la excepción necesaria
from bs4 import BeautifulSoup

In [2]:
headers = {
	"X-RapidAPI-Key": "7aee3d593emshacd060791896a0cp185273jsne6640359eeca",
	"X-RapidAPI-Host": "sofascore.p.rapidapi.com"
}

#### FASE 1: Obtener la url de la pagina web de Sofaescore de todas los partidos de LaLiga 

##### PARTE 1 : Mediante una request a la API de Sofaescore obtenemos el id de todos los equipos de LaLiga para posteriores cnsultas

In [3]:
url = "https://sofascore.p.rapidapi.com/teams/search"

equipos = ["Real Madrid", "Barcelona", "Atlético de Madrid", "Valencia", "Sevilla", "Villarreal", "Real Sociedad", "Real Betis", "Athletic Club", "Celta Vigo", "Almeria", "Getafe", "Mallorca", "Girona", "Granada", "Cadiz", "Alavés", "Rayo Vallecano", "Osasuna", "Las Palmas"]

ids_equipos = []

for equipo in equipos:
    response = requests.get(url, headers=headers, params={"name": equipo})
    data = response.json()
    if data['teams']:
        equipo_id = data['teams'][0]['id']
        ids_equipos.append(equipo_id)
        print(f"GET ID: {equipo}: {equipo_id}")

print(ids_equipos)

GET ID: Real Madrid: 2829
GET ID: Barcelona: 2817
GET ID: Atlético de Madrid: 77879
GET ID: Valencia: 2828
GET ID: Sevilla: 2833
GET ID: Villarreal: 2819
GET ID: Real Sociedad: 2824
GET ID: Real Betis: 2816
GET ID: Athletic Club: 2825
GET ID: Celta Vigo: 2821
GET ID: Almeria: 2858
GET ID: Getafe: 2859
GET ID: Mallorca: 2826
GET ID: Girona: 24264
GET ID: Granada: 33779
GET ID: Cadiz: 4488
GET ID: Alavés: 2885
GET ID: Rayo Vallecano: 2818
GET ID: Osasuna: 2820
GET ID: Las Palmas: 6577
[2829, 2817, 77879, 2828, 2833, 2819, 2824, 2816, 2825, 2821, 2858, 2859, 2826, 24264, 33779, 4488, 2885, 2818, 2820, 6577]


#### PARTE 2 : Mediante una request a la API de Sofaescore obtenemos informacion de todos los equipos de LaLiga

In [7]:
url = "https://sofascore.p.rapidapi.com/teams/get-last-matches"
equipos_data = {}

for team_id in ids_equipos:
    querystring = {"teamId": str(team_id), "pageIndex": "0"}

    response = requests.get(url, headers=headers, params=querystring)

    if response.status_code == 200:
        data = response.json()

        # Almacena la respuesta en el diccionario usando el ID del equipo como clave
        equipos_data[str(team_id)] = data
        print(f"GET info de {team_id}")
    else:
        print(f"Error al obtener datos para el equipo con ID {team_id}. Código de estado: {response.status_code}")

GET info de 2829
GET info de 2817
GET info de 77879
GET info de 2828
GET info de 2833
GET info de 2819
GET info de 2824
GET info de 2816
GET info de 2825
GET info de 2821
GET info de 2858
GET info de 2859
GET info de 2826
GET info de 24264
GET info de 33779
GET info de 4488
GET info de 2885
GET info de 2818
GET info de 2820
GET info de 6577


#### PARTE 3 : Mediante los datos obtenidos de la API cosntruimos la url de cada patido de todos los equipos de LaLiga 

In [8]:
ids_url = []
slugs_url= []
custom_ids_url= []
urls= []

for equipo_id in ids_equipos:
    for i in range(20):
        id = equipos_data[str(equipo_id)]["events"][i]["id"]
        slug = equipos_data[str(equipo_id)]["events"][i]["slug"]
        customId = equipos_data[str(equipo_id)]["events"][i]["customId"]
        
        ids_url.append(id)
        slugs_url.append(slug)
        custom_ids_url.append(customId)

base_url = "https://www.sofascore.com/{}/{}#{}"

for i in range(len(ids_url)):
    url = base_url.format(slugs_url[i], custom_ids_url[i], ids_url[i])
    print(url)
    urls.append(url)
print(urls)

https://www.sofascore.com/real-madrid-chelsea/NsEgb#11137261
https://www.sofascore.com/real-madrid-celta-vigo/wgbsEgb#10408427
https://www.sofascore.com/girona-real-madrid/EgbsoKj#10408437
https://www.sofascore.com/almeria-real-madrid/Egbsihb#10408446
https://www.sofascore.com/real-madrid-real-sociedad/zgbsEgb#10408473
https://www.sofascore.com/real-madrid-osasuna/vgbsEgb#11194439
https://www.sofascore.com/real-madrid-manchester-city/rsEgb#11224240
https://www.sofascore.com/getafe-real-madrid/Egbsjhb#10408479
https://www.sofascore.com/real-madrid-manchester-city/rsEgb#11224737
https://www.sofascore.com/real-madrid-valencia/DgbsEgb#10408258
https://www.sofascore.com/real-madrid-rayo-vallecano/tgbsEgb#10408267
https://www.sofascore.com/sevilla-real-madrid/EgbsIgb#10408309
https://www.sofascore.com/real-madrid-athletic-club/AgbsEgb#10408294
https://www.sofascore.com/real-madrid-milan/RdbsEgb#11327909
https://www.sofascore.com/real-madrid-manchester-united/KEgb#11327936
https://www.sofasco

### FASE 2: "Acceder y hacer scraping de todos las urls de todos los partidos de LaLiga

##### Funcion para extraer todos los datos del performance de un jugador

In [79]:
def obtener_informacion_jugador():

    # Obtiene el contenido HTML de la página
    pagina_html = driver.page_source
    #print(pagina_html)

    # Utiliza BeautifulSoup para analizar el HTML
    soup = BeautifulSoup(pagina_html, 'html.parser')

    print("_______________________________")

    try:
        nombre= driver.find_element(By.XPATH,'//*[@id="__next"]/main/div[3]/div/div/div/div[1]/div/div[1]/a/div')
        print(nombre.text)
        
        try:
            puntuacion= driver.find_element(By.XPATH,'//*[@id="__next"]/main/div[3]/div/div/div/div[1]/div/div[2]/div/span')
            print(puntuacion.text)

            print("_______________________________")

            # DEVOLVER TODOS LOS PARÁMETROS DE RENDIMIENTO DEL JUGADOR: encontrar todos los div con la clase "sc-fqkvVR sc-dcJsrY litZes eFJwJL"
            entradas = []
            goal_elements = soup.find_all('div', class_='sc-fqkvVR sc-dcJsrY litZes eFJwJL')
            for element in goal_elements:
                
                # Encuentra el div con la clase "sc-jEACwC hFGVAX" y el span con la clase "sc-jEACwC jnyhQn" dentro de este div
                div_goal = element.find('div', class_='sc-jEACwC hFGVAX')
                span_goal = element.find('span', class_='sc-jEACwC jnyhQn')
                if div_goal and span_goal:
                    print(div_goal.text)
                    print(span_goal.text)

                    estadisticas = {}
                    clave = div_goal.text
                    valor = span_goal.text
                    estadisticas[clave] = valor

                    entrada = {
                        clave: valor
                    }

                    # Agrega la entrada JSON a la lista
                    entradas.append(entrada)
                    
                nombreJson=nombre.text
                puntuaciónJson=puntuacion.text
        
                # Crear el diccionario para el jugador
                JsonJugador = {
                    nombreJson: {
                        "puntuacion": puntuaciónJson,
                        "estadisticas": entradas
                    }
                }
                
                # Convertir los elementos de la lista a cadena y concatenarlos para formar parte del nombre del archivo
                nombre_archivo = 'performance_jugadores_por_partidos/performance_jugadores_partido_IDPARIDO.json'

                # Verificar si el archivo JSON existe
                if os.path.exists(nombre_archivo):
                    # Si el archivo existe, cargar su contenido
                    with open(nombre_archivo, 'r') as archivo:
                        datos = json.load(archivo)
                else:
                    # Si el archivo no existe, crear un diccionario vacío
                    datos = {}
                
                # Actualizar el diccionario existente con la nueva entrada
                datos.update(JsonJugador)

                # Guardar los datos actualizados en el archivo JSON
                with open(nombre_archivo, 'w') as archivo:
                    json.dump(datos, archivo, indent=4)

        except NoSuchElementException as e:
            print("Sin jugar")
            print("_______________________________")
            
            entradas = []
            estadisticas = {}
            clave = "Minutes played"
            valor = 0
            estadisticas[clave] = valor

            entrada = {
                clave: valor
            }

            # Agrega la entrada JSON a la lista
            entradas.append(entrada)
            
            nombreJson=nombre.text
            puntuaciónJson=None
        
            # Crear el diccionario para el jugador
            JsonJugador = {
                    nombreJson: {
                        "puntuacion": puntuaciónJson,
                        "estadisticas": entradas
                    }
            }
            
            # Convertir los elementos de la lista a cadena y concatenarlos para formar parte del nombre del archivo
            nombre_archivo = 'performance_jugadores_por_partidos/performance_jugadores_partido_IDPARIDO.json'

            # Verificar si el archivo JSON existe
            if os.path.exists(nombre_archivo):
                # Si el archivo existe, cargar su contenido
                with open(nombre_archivo, 'r') as archivo:
                    datos = json.load(archivo)
            else:
                # Si el archivo no existe, crear un diccionario vacío
                datos = {}
                
            # Actualizar el diccionario existente con la nueva entrada
            datos.update(JsonJugador)

            # Guardar los datos actualizados en el archivo JSON
            with open(nombre_archivo, 'w') as archivo:
                json.dump(datos, archivo, indent=4)
            
            return

    except NoSuchElementException as e:
        print("Entrenador")
    print("_______________________________")

##### PARTE 1 : obtener el performance de los 22 jugadores titulares del partido

###### (abre la web, acepta las cookies, hace click sobre de cada jugador para que emerja la tarjeta con los datos del performance del partido asociados a cada jugador y los extrae)

In [58]:
for i in range(22):
    # Crea una instancia del controlador del navegador
    driver = webdriver.Chrome()

    # Navega a la página web que deseas hacer scraping
    driver.get("https://www.sofascore.com/real-madrid-chelsea/NsEgb#11137259")

    # Espera a que se cargue la página
    driver.implicitly_wait(45)

    # Encuentra el botón de "Consentir" 
    button = driver.find_element(By.XPATH, '//button[@aria-label="Consentir"]')
    # Haz clic en el botón de "Consentir" 
    button.click()

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # Encuentra todos los elementos <a> con la clase 'sc-3937c22d-0 jrbLdB'
    divJugadores = driver.find_elements(By.XPATH, '//a[@class="sc-3937c22d-0 jrbLdB"]')

    divJugadores[i].click()
    time.sleep(45)
    obtener_informacion_jugador()

    driver.quit()

_______________________________
Thibaut Courtois
7.5
_______________________________
Minutes played
90'
Saves
3
Goals prevented
1.38
Punches
0
Runs out (succ.)
0 (0)
High claims
0
Saves from inside box
3
Touches
23
Acc. passes
12/13 (92%)
Key passes
0
Crosses (acc.)
0 (0)
Long balls (acc.)
5 (4)
Clearances
0
Blocked shots
0
Interceptions
0
Tackles
0
Dribbled past
0
Ground duels (won)
0 (0)
Aerial duels (won)
0 (0)
Fouls
0
Was fouled
0
Shots on target
0
Shots off target
0
Shots blocked
0
Dribble attempts (succ.)
0 (0)
_______________________________
_______________________________
Daniel Carvajal
7.8
_______________________________
Minutes played
90'
Goals
0
Assists
0
Expected Assists (xA)
0.46
Clearances
1
Blocked shots
1
Interceptions
1
Tackles
1
Dribbled past
1
Ground duels (won)
4 (2)
Aerial duels (won)
2 (0)
Possession lost
9
Fouls
1
Was fouled
1
Touches
82
Acc. passes
55/60 (92%)
Key passes
2
Crosses (acc.)
3 (0)
Long balls (acc.)
3 (2)
Big chances created
1
Shots on target
0
Shot

##### PARTE 2 :  obtener el performance de los jugadores que entraron de cambio al partido

In [81]:
jugador=0

while True:
    
    if jugador >= len(divJugadores):
        break
        
    # Crea una instancia del controlador del navegador
    driver = webdriver.Chrome()

    # Navega a la página web que deseas hacer scraping
    driver.get("https://www.sofascore.com/real-madrid-chelsea/NsEgb#11137259")

    # Espera a que se cargue la página
    driver.implicitly_wait(35)

    #Encuentra el botón de "Consentir" 
    button = driver.find_element(By.XPATH, '//button[@aria-label="Consentir"]')
    # Haz clic en el botón de "Consentir" 
    button.click()

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # Encuentra todos los elementos 
    divJugadores = driver.find_elements(By.XPATH, '//div[@display="flex" and contains(@class, "sc-fqkvVR sc-dcJsrY bASBgQ kHiXJe")]')
    
    tamaño_divJugadores = len(divJugadores)
    print(jugador,"/", tamaño_divJugadores)

    divJugadores[jugador].click()
    time.sleep(35)
    obtener_informacion_jugador()
    
    jugador += 1 
           
    driver.quit()

0 / 24
_______________________________
Entrenador
_______________________________
1 / 24
_______________________________
Antonio Rüdiger
6.8
_______________________________
Minutes played
19'
Goals
0
Assists
0
Clearances
1
Blocked shots
1
Interceptions
0
Tackles
1
Dribbled past
0
Ground duels (won)
1 (1)
Aerial duels (won)
0 (0)
Possession lost
1
Fouls
0
Was fouled
0
Touches
8
Acc. passes
4/5 (80%)
Key passes
0
Crosses (acc.)
0 (0)
Long balls (acc.)
0 (0)
Shots on target
0
Shots off target
0
Shots blocked
0
Dribble attempts (succ.)
0 (0)
_______________________________
2 / 24
_______________________________
Marco Asensio
7.4
_______________________________
Minutes played
19'
Goals
1
Expected Goals (xG)
0.08
Assists
0
Expected Assists (xA)
0.02
Shots on target
1
Shots off target
1
Shots blocked
0
Dribble attempts (succ.)
0 (0)
Touches
17
Acc. passes
10/11 (91%)
Key passes
0
Crosses (acc.)
0 (0)
Long balls (acc.)
0 (0)
Ground duels (won)
0 (0)
Aerial duels (won)
0 (0)
Possession lost
1
F